# 순환 신경망
## 순환 신경망의 내부 구조

In [1]:
import torch
import torch.nn as nn

class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True):
        super(CustomRNN, self).__init__()
        
        # 선형결합에 사용할 학습가능한 매개변수 생성
        self.weight_xh, self.weight_hh, self.bias = \
            self.init_weight(input_size, hidden_size)
        
        # 필요 정보 저장
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        
    def forward(self, inputs):
        """
        rnn_cell 구동하기 위해 inputs의 크기가 (T,B,E) 형태 되야함.
        - T: 시퀀스 총 길이
        - B: 미니배치 크기
        - E: 입력층 크기
        """
        if self.batch_first:
            # 첫번째 차원이 미니배치 크기인 경우 전치연산으로 바꿔줌 
            inputs = inputs.transpose(0,1)
            
        seqlen, batch_size, _ = inputs.size()
        
        # 0 time-step에서 은닉층 값을 0으로 초기화
        hidden = self.init_hidden(batch_size, self.hidden_size)
        
        # output에 은닉층의 출력값 저장
        output = []
        
        # 시퀀스의 총 길이만큼 순방향 전파 진행
        for i in range(seqlen):
            hidden = self.rnn_cell(inputs[i], hidden)
            output.append(hidden)
        
        output = torch.stack(output)
        
        if self.batch_first:
            output = output.transpose(0,1)
            
        # 모든 타임스텝의 은닉층 출력값과 마지막 타임스텝의 은닉층 출력값을 각각 반환
        return output, hidden
    
    def rnn_cell(self, x, h):
        """ RNN Cell """
        h = x.mm(self.weight_xh.t()) + h.mm(self.weight_hh.t()) + self.bias
        return torch.tanh(h)
    
    def init_hidden(self, batch_size, hidden_size):
        """ 0 타입스텝에서 은닉층 초기화 """
        return torch.zeros(batch_size, hidden_size)
    
    def init_weight(self, input_size, hidden_size):
        """ rnn_cell 의 선형결합을 위한 초기값 """
        weight_xh = torch.randn(hidden_size, input_size).requires_grad_()
        weight_hh = torch.randn(hidden_size, hidden_size).requires_grad_()
        bias = torch.zeors(1, hidden_size).requires_grad_()
        
        return weight_xh, weight_hh, bias
        

## 순환 신경망의 단점 : 장기 의존성 문제 (Long-Term Dependency)
- 타임스텝이 길어질수록 예전에 있던 정보를 기억하지 못한다.
- --> 미분값이 0에 수렴하여 역전파가 앞단까지 전달되지 않는다.
- ex. "나는 한국 사람이고, **중국 칭다오에 아주 오래 살다가** 지금은 서울에 살고 있어. ... 어쨎든, 미국으로 유학을 다녀왔지만 영어는 잘 못해, 하지만 **중국어는 잘해!**"
- "경사 소실": 긴 시퀀스의 경우 미분값이 0에 가까워져 매개변수에 반영되지 않음.

## LSTM (Long Short-term Memory)
- RNN Cell 안에 별도의 Cell State 변수 만들어 경사 소실 문제 피하려 함.
- Cell State는 각 타임스텝에서 기억할 정보와 망각할 정보를 처리, 향후 경사를 잘 전달할 수 있게 만드는 통로 역할을 함. (= 컨베이어 벨트)
- 많은 매개변수 필요함. 

## GRU (Gated Recurrent Unit)
- LSTM 보다 좀더 간단하고 비슷한 성능 냄.
- 0~1 사이 게이트가 정보 유지할지 잊을지 결정.

아직까지는 GRU보단 LSTM이 더 많이 쓰이고 있음. 취향 차이

In [2]:
# RNN, LSTM, GRU 호출 방법

rnn_layer = nn.RNN(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)

rnn_layer = nn.LSTM(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)

rnn_layer = nn.GRU(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)

RNN(5, 10, batch_first=True)
LSTM(5, 10, batch_first=True)
GRU(5, 10, batch_first=True)


## 간단한 순환 신경망 예제
- "We are going to watch Avengers End Game" 출력하는 모델 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(70)

sentence = "We are going to watch Avengers End Game".split()
vocab = {token: i for i, token in enumerate(sentence, 1)}
vocab['<unk>'] = 0

# 수치화된 데이터를 단어로 바꾸기 위한 사전 
rev_vocab = {v:k for k,v in vocab.items()}

# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [rev_vocab.get(x) for x in y]

def construct_data(sentence, vocab):
    """
    (input, target) 쌍으로 데이터 생성
    - 최종 형태 (수치화된 단어 쌍):
      [(We, are), (are, going), (going, to), (to, watch),
          (watch, Avengers), (Avengers, End), (End, Game)]
    """
    numericalize = lambda x: vocab.get(x) if vocab.get(x) is not None else 0
    totensor = lambda x: torch.LongTensor(x)
    idxes = [numericalize(token) for token in sentence]
    x,t = idxes[:-1], idxes[1:]
    
    return totensor(x).unsqueeze(0), totensor(t).unsqueeze(0)

class Net(nn.Module):
    """ 예제 문장을 출력하는 모델 """
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        """
        vocab_size : 단어장 크기
        input_size : 임베딩 크기 (RNN 입력층 크기)
        hidden_size : RNN 은닉층 크기
        """
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size,
                               batch_first = batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x):
        """
        텐서 크기 변화 위한 문자 설명
        - V: 단어장 크기
        - T: 시퀀스 총 길이
        - B: 미니배치 크기
        - E: 임베딩 크기 (RNN 입력층 크기)
        - D: RNN 은닉층 크기
        """
        
        # 1. embedding 층 통과해서 분산표상 방식으로 단어 표현
        # 크기변화: (B,T) > (B,T,D)
        output = self.embedding_layer(x)
        
        # 2. RNN 층
        # 크기변화: (B,T,D) > output (B,T,D) / hidden (1,B,D)
        output, hidden = self.rnn_layer(output)
        
        # 3. 최종 출력층
        # 크기변화: (B,T,D) > (B,T,V)
        output = self.linear(output)
        
        # 4. 모델 출력:
        # 크기변화: (B,T,V) > (B*T, V)
        return output.view(-1, output.size(2))
    

# 데이터 생성
x, t = construct_data(sentence, vocab)

# 모델 생성을 위한 하이퍼파라미터 설정
vocab_size = len(vocab)  # 임베딩 층, 최종 출력층에 사용될 단어장 크기
input_size = 5   # 임베딩 차원 크기(RNN 입력 차원 크기)
hidden_size = 20 # RNN 은닉층 크기

# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실함수 정의 
loss_function = nn.CrossEntropyLoss()

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

# 훈련 시작
for step in range(151):
    # 1) 경사 초기화
    optimizer.zero_grad()
    # 2) 순방향 전파
    output = model(x)
    # 3) 손실값 계산
    loss = loss_function(output, t.view(-1))
    # 4) 역방향 전파
    loss.backward()
    # 5) 매개변수 업데이트
    optimizer.step()
    # 기록 
    if step % 30 == 0:
        print("[{:02d}/151] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(['We'] + decode(pred)))
        print()

[01/151] 2.3034 
We Avengers watch are going watch watch Avengers

[31/151] 1.9172 
We are Avengers going are Avengers End Game

[61/151] 1.5057 
We are going going watch Avengers End Game

[91/151] 1.0342 
We are going to watch Avengers End Game

[121/151] 0.6438 
We are going to watch Avengers End Game

[151/151] 0.3932 
We are going to watch Avengers End Game



In [4]:
vocab

{'We': 1,
 'are': 2,
 'going': 3,
 'to': 4,
 'watch': 5,
 'Avengers': 6,
 'End': 7,
 'Game': 8,
 '<unk>': 0}

In [5]:
rev_vocab

{1: 'We',
 2: 'are',
 3: 'going',
 4: 'to',
 5: 'watch',
 6: 'Avengers',
 7: 'End',
 8: 'Game',
 0: '<unk>'}

In [9]:
output

tensor([[-0.7617, -0.6662,  2.6440,  0.3793, -0.8535,  0.7474,  0.9093, -0.5351,
         -0.6034],
        [-0.6160, -0.6983,  0.0350,  2.6441,  0.5980, -0.9655, -0.3289,  1.0526,
         -0.6033],
        [-0.5833, -0.7808, -0.8670,  0.5197,  2.4368, -0.2169, -2.5131, -0.3115,
          0.8703],
        [-0.8453, -0.5904,  0.2934, -0.7246, -0.4353,  2.7860,  0.3800, -2.3972,
          0.1200],
        [-0.6625, -0.5480,  1.3101, -0.0367, -1.8888,  1.1051,  3.3165,  0.1471,
         -0.4979],
        [-0.3904, -0.3850, -0.2528,  1.1922,  0.3245, -1.6094,  0.4992,  3.2076,
          0.9398],
        [-1.7164, -0.6710, -0.5265, -0.3158,  0.7102, -0.2211, -0.9152,  0.5260,
          2.9876]], grad_fn=<ViewBackward>)

In [10]:
output.softmax(0)

tensor([[0.1387, 0.1354, 0.6413, 0.0644, 0.0238, 0.0897, 0.0723, 0.0184, 0.0198],
        [0.1604, 0.1311, 0.0472, 0.6202, 0.1016, 0.0162, 0.0209, 0.0900, 0.0198],
        [0.1658, 0.1208, 0.0192, 0.0741, 0.6390, 0.0342, 0.0024, 0.0230, 0.0865],
        [0.1276, 0.1461, 0.0611, 0.0214, 0.0362, 0.6890, 0.0426, 0.0029, 0.0408],
        [0.1531, 0.1524, 0.1689, 0.0425, 0.0085, 0.1283, 0.8023, 0.0364, 0.0220],
        [0.2010, 0.1794, 0.0354, 0.1452, 0.0773, 0.0085, 0.0479, 0.7763, 0.0927],
        [0.0534, 0.1348, 0.0269, 0.0321, 0.1137, 0.0341, 0.0117, 0.0531, 0.7184]],
       grad_fn=<SoftmaxBackward>)

In [11]:
output.softmax(1)

tensor([[0.0206, 0.0227, 0.6208, 0.0645, 0.0188, 0.0932, 0.1095, 0.0258, 0.0241],
        [0.0240, 0.0221, 0.0461, 0.6261, 0.0809, 0.0169, 0.0320, 0.1275, 0.0243],
        [0.0301, 0.0247, 0.0226, 0.0906, 0.6162, 0.0434, 0.0044, 0.0395, 0.1287],
        [0.0192, 0.0248, 0.0600, 0.0217, 0.0289, 0.7255, 0.0654, 0.0041, 0.0504],
        [0.0135, 0.0151, 0.0969, 0.0252, 0.0040, 0.0789, 0.7203, 0.0303, 0.0159],
        [0.0188, 0.0189, 0.0216, 0.0917, 0.0385, 0.0056, 0.0458, 0.6878, 0.0712],
        [0.0067, 0.0191, 0.0221, 0.0272, 0.0760, 0.0299, 0.0150, 0.0632, 0.7408]],
       grad_fn=<SoftmaxBackward>)

In [8]:
output.softmax(-1)

tensor([[0.0206, 0.0227, 0.6208, 0.0645, 0.0188, 0.0932, 0.1095, 0.0258, 0.0241],
        [0.0240, 0.0221, 0.0461, 0.6261, 0.0809, 0.0169, 0.0320, 0.1275, 0.0243],
        [0.0301, 0.0247, 0.0226, 0.0906, 0.6162, 0.0434, 0.0044, 0.0395, 0.1287],
        [0.0192, 0.0248, 0.0600, 0.0217, 0.0289, 0.7255, 0.0654, 0.0041, 0.0504],
        [0.0135, 0.0151, 0.0969, 0.0252, 0.0040, 0.0789, 0.7203, 0.0303, 0.0159],
        [0.0188, 0.0189, 0.0216, 0.0917, 0.0385, 0.0056, 0.0458, 0.6878, 0.0712],
        [0.0067, 0.0191, 0.0221, 0.0272, 0.0760, 0.0299, 0.0150, 0.0632, 0.7408]],
       grad_fn=<SoftmaxBackward>)

In [12]:
output.softmax(-1).argmax(-1)

tensor([2, 3, 4, 5, 6, 7, 8])

In [13]:
output.softmax(-1).argmax(-1).tolist()

[2, 3, 4, 5, 6, 7, 8]

In [14]:
construct_data(sentence, vocab)

(tensor([[1, 2, 3, 4, 5, 6, 7]]), tensor([[2, 3, 4, 5, 6, 7, 8]]))

In [15]:
vocab_size

9

In [16]:
torch.tensor([[2, 3, 4, 5, 6, 7, 8]]).view(-1)

tensor([2, 3, 4, 5, 6, 7, 8])